In [1]:
%%capture
!pip install nfl_data_py
!pip install cfbd

In [2]:
#Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import nfl_data_py as nfl
from collections import Counter
import requests
from bs4 import BeautifulSoup
import re
import cfbd
from scipy import stats
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

In [3]:
# configuration = cfbd.Configuration()
# configuration.api_key['Authorization'] = '5Hs/5kCfcbyDhcVBb98O+KQdxKfJsdG1QbYUlwGYp+ncYyNptbvHzsSz/r71nvCN'
# configuration.api_key_prefix['Authorization'] = 'Bearer'
# playersAPI = cfbd.PlayersApi(cfbd.ApiClient(configuration))

In [4]:
#nfl.see_weekly_cols()
WEEKS = 18
def weekly_qbr(row):
    a = ((row['completions'] / row['attempts']) - 0.3) * 5
    b = ((row['passing_yards'] / row['attempts']) - 3) * 0.25
    c = (row['passing_tds'] / row['attempts']) * 20
    d = 2.375 - ((row['interceptions'] / row['attempts']) * 25)
    ret = ((a + b + c + d) / 6) * 100
    return(ret)

def yearly_total_qbr(row):
    a = ((row['yearly_completions'] / row['yearly_attempts']) - 0.3) * 5
    b = ((row['yearly_passing_yards'] / row['yearly_attempts']) - 3) * 0.25
    c = (row['yearly_passing_tds'] / row['yearly_attempts']) * 20
    d = 2.375 - ((row['yearly_interceptions'] / row['yearly_attempts']) * 25)
    ret = ((a + b + c + d) / 6) * 100
    return(ret)

#Getting weekly stats and seasonal stats
seasonal = nfl.import_seasonal_data(list([2022]), s_type='REG')
seasonal = seasonal[['player_id', 'attempts', 'completions', 'passing_yards', 'passing_tds', 'interceptions']]
seasonal.columns = ['player_id', 'yearly_attempts', 'yearly_completions', 'yearly_passing_yards', 'yearly_passing_tds', 'yearly_interceptions']
data = nfl.import_weekly_data(list([2022]), list(['player_id', 'player_display_name', 'position', 'week', 'attempts', 'completions', 'passing_yards', 'passing_tds', 'interceptions']))
data = data[(data['attempts'] >= 5) & (data['position'] == 'QB') & (data['week'] <= 18)]

#Weekly QBRs
data['weekly_qbr'] = data.apply(lambda x: weekly_qbr(x), axis = 1)
data = data.merge(seasonal.reset_index(drop=True), how = 'left', on = ['player_id'])
data = data[(data['yearly_attempts'] >= 50)]

#Calculating QBRs
data['yearly_total_qbr'] = data.apply(lambda x: yearly_total_qbr(x), axis = 1)
qbs = data.groupby('player_id')['weekly_qbr'].mean()
data = data.merge(qbs, how = 'left', on = ['player_id'])
data = data.rename(columns = {'weekly_qbr_y': 'average_qbr'})

#Prepping and calculating final values
data['average_qbr'] = data['average_qbr'].round(3)
data['yearly_total_qbr'] = data['yearly_total_qbr'].round(3)
data.drop_duplicates(subset = ['player_id'], inplace = True)
data['consistency'] = (data['average_qbr'] - data['yearly_total_qbr']).round(3)
data.sort_values(['consistency'], ascending = False, inplace = True)
data = data.reset_index()
data = data[['player_display_name', 'yearly_total_qbr', 'average_qbr', 'consistency']]
data

Downcasting floats.


,player_display_name,yearly_total_qbr,average_qbr,consistency
0,Mike White,75.702,83.970,8.268
1,Sam Ehlinger,76.052,79.999,3.947
2,Matthew Stafford,87.438,91.103,3.665
3,Daniel Jones,92.523,95.892,3.369
4,Deshaun Watson,79.093,82.252,3.159
5,Jacoby Brissett,88.895,92.053,3.158
6,Marcus Mariota,88.181,91.316,3.135
7,Mitchell Trubisky,81.065,84.091,3.026
8,Cooper Rush,80.041,83.033,2.992
9,Tyler Huntley,77.158,79.992,2.834


In [5]:
data.to_csv('ConsistencyRankings')